In [ ]:
#https://www.machinelearningplus.com/nlp/topic-modeling-gensim-python/#1introduction
!pip install gensim
!pip install pyLDAvis
!pip install nltk
import numpy as np
import json
import glob
import pandas
pandas.set_option('display.max_colwidth', None)
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer



#Gensim
import gensim
import gensim.corpora as corpora
from gensim import models
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
from gensim.models import Phrases
from gensim.models.phrases import Phraser


#spacy
import spacy
from nltk.corpus import stopwords

#vis
import pyLDAvis
import pyLDAvis.gensim_models

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [ ]:
def add_hours(minDate:str,maxDate:str)-> (str,str):
	minDate = minDate + "00:00:00"
	maxDate = maxDate + "23:59:59"

	return minDate,maxDate

def get_comments(originPath:str,minDate:str = None,maxDate:str = None,versions = None) -> pandas.DataFrame:
	
	commentsFile = pandas.read_csv(originPath)

	if minDate != None and maxDate != None:
		minDate,maxDate = add_hours(minDate,maxDate)

	for i in range(0, len(commentsFile)):
		if commentsFile['date'][i] < minDate or commentsFile['date'][i] > maxDate or commentsFile['version'][i] not in versions:
			#print("droped",commentsFile['version'][i])
			commentsFile = commentsFile.drop([i])
	return commentsFile

def lemmatization(texts, allowed_postags):
    nlp = spacy.load("pt_core_news_sm")
    texts_out = []
    for text in texts:
        doc = nlp(text)
        new_text = []
        for token in doc:
            if token.pos_ in allowed_postags:
                new_text.append(token.lemma_)
        final = " ".join(new_text)
        texts_out.append(final)
    return (texts_out)

def gen_words(texts):
    final = []
    for text in texts:
        new = gensim.utils.simple_preprocess(text, deacc=True)
        final.append(new)
    return (final)

In [ ]:
database = get_comments("./General_Data.csv","2021-01-01","2021-10-20","70.4.0")
data = database['content']

In [ ]:
score1 = database[database['score'] == 1]
data = score1['content'] # selectionando apenas os comentários com nota = 1

In [ ]:
lemmatized_texts = lemmatization(data, ["NOUN", "VERB", "NUM", "ADV", "ADJ"])
print (lemmatized_texts[0])

In [ ]:
data_words = gen_words(lemmatized_texts)
print (data_words[0])

# Extraindo tópicos a partir de palavras unitárias (1-gram)¶

In [ ]:
id2word = corpora.Dictionary(data_words)

corpus = []
for text in data_words:
    new = id2word.doc2bow(text)
    corpus.append(new)

print (corpus[0])

word = id2word[[7][:1][0]]
print (word)

In [ ]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=10,
                                           random_state=1000,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha="auto")

In [ ]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word, mds="mmds", R=100)
vis

# Extraindo tópicos em n-grams

In [25]:
N = 3 #qtde de ngrams
print(data_words[0])
ngrams = []
for sentence in data_words:
    ngrams_temp = []
    tokens = list(sentence)
    for i in range(0,len(tokens)-(N-1)):
        new_text = tokens[i]
        for k in range(1,N):
            new_text = new_text + '_' + tokens[i+k]
        ngrams_temp.append(new_text)
    ngrams.append(ngrams_temp)

['cadastrar', 'aplicativo', 'pedir', 'mail', 'dizer', 'invalido', 'precisar', 'posicao']


In [26]:
id2ngram = corpora.Dictionary(ngrams)

In [27]:
corpus_ngram = []
for text in ngrams:
    new = id2ngram.doc2bow(text)
    corpus_ngram.append(new)

print (corpus_ngram[0])

[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1)]


In [28]:
lda_model_ngram = gensim.models.ldamodel.LdaModel(corpus=corpus_ngram,
                                           id2word=id2ngram,
                                           num_topics=5,
                                           random_state=1000,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha="auto")

In [29]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model_ngram, corpus_ngram, id2ngram, mds="mmds", R=100)
vis

/srv/conda/envs/notebook/lib/python3.7/site-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3      0.174034  0.037971       1        1  36.567108
2     -0.006407  0.018804       2        1  16.445973
4     -0.088249 -0.038415       3        1  16.266633
0     -0.077178  0.074856       4        1  15.915361
1     -0.002201 -0.093216       5        1  14.804925, topic_info=                          Term      Freq     Total Category   logprob  \
481         nao_conseguir_mais  3.000000  3.000000  Default  100.0000   
140    nao_conseguir_cadastrar  2.000000  2.000000  Default   99.0000   
161         opcao_criar_contar  1.000000  1.000000  Default   98.0000   
139              nao_ter_opcao  2.000000  2.000000  Default   97.0000   
745  atualizacao_nao_conseguir  1.000000  1.000000  Default   96.0000   
..                         ...       ...       ...      ...       ...   
68         nao_conseguir_fazer  0.371185  2.545523   Topic5   -6.3078   
602          nao_funcionar_nao  0.369915  2.205381   Topic5   -6.3112   
63     cadastrar_nao_conseguir  0.369896  1.102916   Topic5   -6.3113   
618          ficar_tela_branco  0.369894  1.698345   Topic5   -6.3113   
283      nao_conseguir_acessar  0.369894  1.067437   Topic5   -6.3113   

      loglift  
481  100.0000  
140   99.0000  
161   98.0000  
139   97.0000  
745   96.0000  
..        ...  
68    -0.0152  
602    0.1248  
63     0.8177  
618    0.3860  
283    0.8504  

[652 rows x 6 columns], token_table=      Topic      Freq                         Term
term                                              
1258      5  0.656651             abrir_muito_ruim
294       4  0.923158  acesso_certificar_vacinacao
1100      5  0.656652             acesso_fazer_ter
1130      1  0.720280              achar_errar_app
1131      1  0.720280             achar_pessoa_nao
...     ...       ...                          ...
1177      1  0.720280              util_funcao_app
1055      1  0.720280         util_querer_aparecer
1056      1  0.720280             vacinar_covid_ja
1178      1  0.720280       vacinar_dever_gratuito
1179      1  0.720280             ver_entrar_locar

[182 rows x 3 columns], R=100, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 3, 5, 1, 2])